In [3]:
import h5py
import numpy as np
import pandas as pd
import os
import pandas as pd
import seaborn as sn
import torch
import torchvision.models as models
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from IPython.core.display import display
from collections.abc import Mapping
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
import torchmetrics
from torchvision.transforms import ToTensor

/tmp/ipykernel_94604/786869960.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [37]:
x_keys = ['l', 'b', 'pmra', 'pmdec']
y_keys = ['is_accreted']
path = '/scratch/08949/tg882489/data/Ananke/Ananke_data_reduced.hdf5'

with h5py.File(path, 'r') as f:
    x = []
    for k in range(0, len(x_keys)):
        x.append(f[x_keys[k]][:])
        print(k/len(x_keys))
    y = f['is_accreted'][:]


x = np.vstack(x).T
shuffle = np.random.permutation(len(x))
x = x[shuffle]
y = y[shuffle]

n_test = int(0.1 *len(x))
test_x = x[:n_test]
test_y = y[:n_test]

#save shuffle n_train , n_val

ny1 = np.sum(test_y==1)
ny0 = np.sum(test_y==0)
ny = ny1 + ny0

w1 = ny/ny1
w0 = ny/ny0
#weight = torch.tensor([w0, w1], dtype=torch.float32)

mean_test_x = np.mean(test_x, axis = 0)
stdv_test_x = np.std(test_x, axis = 0)

test_x = (test_x - mean_test_x) / stdv_test_x

test_x = torch.tensor(test_x, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.long)

test_dataset = list(zip(test_x, test_y))
test_loader = DataLoader(test_dataset, batch_size = 1024)

0.0
0.25
0.5
0.75


SystemError: <method 'unbind' of 'torch._C._TensorBase' objects> returned a result with an exception set

In [23]:
class Model(LightningModule):
    def __init__(self):
        super().__init__() #init: call to the parent class must be made before assignment on the child
        self.l1 = torch.nn.Linear(4, 100) #nn.linear: Applies a linear transformation to the incoming data
        self.l2 = torch.nn.Linear(100, 50)
        self.l3 = torch.nn.Linear(50, 2)
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        #self.weight = weight

    def forward(self, x):
        x_out = self.l1(x)
        x_out = torch.relu(x_out)
        x_out = self.l2(x_out)
        x_out = torch.relu(x_out)
        x_out = self.l3(x_out)
        return x_out
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3) #Call an optimizer and define learning rate
    
    def training_step(self, batch, batch_nb):
        train_x, train_y = batch 
        preds = self(train_x)
        loss = F.cross_entropy(preds, train_y, weight = self.weight)
        self.train_acc(preds, train_y)
        self.log('train_loss', loss)
        self.log('train_acc', self.train_acc, on_step=False, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_nb):
        val_x, val_y = batch
        preds = self(val_x)
        loss = F.cross_entropy(preds, val_y, weight=self.weight)
        self.valid_acc(preds, val_y)
        self.log('val_loss', loss)
        self.log('valid_acc', self.valid_acc, on_step=False, on_epoch=True)
        return loss

In [24]:
#Loading in the model

model = Model.load_from_checkpoint('/scratch/08949/tg882489/results/Ananke_logs/version_1/checkpoints/best.ckpt')

'/scratch/08949/tg882489/results/Ananke_logs/version_1/checkpoints/best.ckpt'

In [25]:
model

<bound method ModelIO.load_from_checkpoint of <class '__main__.Model'>>

In [28]:
len(test_loader)

4950

In [38]:
predict = []
target = []
with torch.no_grad():
    for batch in test_loader:
        test_x, test_y = batch
        test_x = test_x.view(-1)
        yhat = model(test_x)
        predict.append(yhat.cpu().numpy())
        target.append(y.cpu().numpy())
predict = np.concatenate(predict)
target = np.concatenate(target)

AttributeError: 'Tensor' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [31]:
def nn(test_x)
    l1 = torch.nn.Linear(4, 100)
    l2 = torch.nn.Linear(100, 50)
    l3 = torch.nn.Linear(50, 2)
    x_out = l1(test_x)
    x_out = torch.relu(x_out)
    x_out = l2(x_out)
    x_out = torch.relu(x_out)
    x_out = l3(x_out)
    return x_out

In [32]:
x_out

tensor([[ 0.0496,  0.0968],
        [ 0.0735,  0.0591],
        [ 0.0252,  0.0847],
        ...,
        [ 0.1021, -0.0871],
        [-0.0234, -0.0132],
        [ 0.0328,  0.0438]], grad_fn=<AddmmBackward0>)

In [26]:
trainer = Trainer()
trainer.test(model, dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


TypeError: Unwrapping the module did not yield a `LightningModule`, got <class 'method'> instead.

In [13]:
class ClassificationAcc:
    def __init__(self, preds, labels, num_classes):
        super().__init__()
        self.preds = np.array(preds)
        self.labels = np.array(labels)
        self.num_classes = num_classes
        self.count_matrix = np.zeros((self.num_classes, self.num_classes), dtype=np.int32)
        
        for i in range(len(self.preds)):
            self.count_matrix[self.labels[i]][self.preds[i]]+=1
        self.recall_matrix = self.count_matrix / np.sum(self.count_matrix, axis=1)[...,None]
        self.precision_matrix = self.count_matrix / np.sum(self.count_matrix, axis-0)[...,None]
        

Model(
  (l1): Linear(in_features=4, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=50, bias=True)
  (l3): Linear(in_features=50, out_features=2, bias=True)
  (train_acc): Accuracy()
  (valid_acc): Accuracy()
)